In [1]:
import tools as t
from tools.imports import *
from copy import deepcopy
from tools.utils import n2

In [2]:
a, b       = t.normalize([1+2j, 2+3j])
c, d, e, f = t.normalize([3+4j, 1+4j, 5, 6])
alligators = [a, b]
swim = [c, d, e, f]

In [3]:
# hadamard real part

qr = qk.QuantumRegister(4, 'q')
cr = qk.ClassicalRegister(2, 'c')
qc = qk.QuantumCircuit(qr, cr)

qc.initialize(alligators, 1)
qc.initialize(swim, [3, 2])

qc.barrier()

qc.h(0)
qc.cswap(0, 1, 2)
qc.h(0)

qc.barrier()

qc.measure(0, 0)
qc.measure(3, 1)

print(qc.draw())

qc_real = deepcopy(qc)

                                                                       ░ ┌───┐»
q_0: ──────────────────────────────────────────────────────────────────░─┤ H ├»
               ┌────────────────────────────────────────────┐          ░ └───┘»
q_1: ──────────┤ Initialize(0.2357+0.4714j,0.4714+0.70711j) ├──────────░──────»
     ┌─────────┴────────────────────────────────────────────┴────────┐ ░      »
q_2: ┤1                                                              ├─░──────»
     │  Initialize(0.2956+0.39413j,0.098533+0.39413j,0.49266,0.5912) │ ░      »
q_3: ┤0                                                              ├─░──────»
     └───────────────────────────────────────────────────────────────┘ ░      »
c: 2/═════════════════════════════════════════════════════════════════════════»
                                                                              »
«        ┌───┐ ░ ┌─┐   
«q_0: ─■─┤ H ├─░─┤M├───
«      │ └───┘ ░ └╥┘   
«q_1: ─X───────░──╫────
«      │       ░  ║    


In [4]:
# hadamard real part

qr = qk.QuantumRegister(4, 'q')
cr = qk.ClassicalRegister(2, 'c')
qc = qk.QuantumCircuit(qr, cr)

qc.initialize(alligators, 1)
qc.initialize(swim, [3, 2])

qc.barrier()

qc.h(0)
qc.sdg(0)
qc.cswap(0, 1, 2)
qc.h(0)

qc.barrier()

qc.measure(0, 0)
qc.measure(3, 1)

print(qc.draw())

qc_imag = deepcopy(qc)

                                                                       ░ ┌───┐»
q_0: ──────────────────────────────────────────────────────────────────░─┤ H ├»
               ┌────────────────────────────────────────────┐          ░ └───┘»
q_1: ──────────┤ Initialize(0.2357+0.4714j,0.4714+0.70711j) ├──────────░──────»
     ┌─────────┴────────────────────────────────────────────┴────────┐ ░      »
q_2: ┤1                                                              ├─░──────»
     │  Initialize(0.2956+0.39413j,0.098533+0.39413j,0.49266,0.5912) │ ░      »
q_3: ┤0                                                              ├─░──────»
     └───────────────────────────────────────────────────────────────┘ ░      »
c: 2/═════════════════════════════════════════════════════════════════════════»
                                                                              »
«     ┌─────┐   ┌───┐ ░ ┌─┐   
«q_0: ┤ Sdg ├─■─┤ H ├─░─┤M├───
«     └─────┘ │ └───┘ ░ └╥┘   
«q_1: ────────X───────░──╫─

In [5]:
s_real = t.submit(qc_real, shots=2**19)
s_imag = t.submit(qc_imag, shots=2**19)

r_real = t.results(s_real.counts, s_real.shots).counts
r_imag = t.results(s_imag.counts, s_imag.shots).counts

print('r_real counts ', r_real)
print('r_imag counts ', r_imag)

r_real counts  {'10': 24085, '11': 36132, '00': 230152, '01': 233919}
r_imag counts  {'11': 134815, '10': 127599, '00': 126965, '01': 134909}


In [6]:
def cenas(dic):
    dic_list = [{}, {}]
    for key, value in dic.items():
        q0, q3 = key
        dic_list[int(q3)][q0] = dic_list[int(q3)].get(q0, 0) + value
    return dic_list

In [7]:
counts_real_q3_0, counts_real_q3_1 = cenas(r_real)
print('counts_real for q_3 = 0: ', counts_real_q3_0)
print('counts_real for q_3 = 1: ', counts_real_q3_1)
print('\n')
counts_imag_q3_0, counts_imag_q3_1 = cenas(r_imag)
print('counts_imag for q_3 = 0: ', counts_imag_q3_0)
print('counts_imag for q_3 = 1: ', counts_imag_q3_1)

counts_real for q_3 = 0:  {'1': 24085, '0': 230152}
counts_real for q_3 = 1:  {'1': 36132, '0': 233919}


counts_imag for q_3 = 0:  {'1': 127599, '0': 126965}
counts_imag for q_3 = 1:  {'1': 134815, '0': 134909}


In [8]:
expect_value_real_q3_0 = (counts_real_q3_0['0'] - counts_real_q3_0['1'])/s_real.shots
expect_value_real_q3_1 = (counts_real_q3_1['0'] - counts_real_q3_1['1'])/s_real.shots
expect_value_imag_q3_0 = (counts_imag_q3_0['0'] - counts_imag_q3_0['1'])/s_imag.shots
expect_value_imag_q3_1 = (counts_imag_q3_1['0'] - counts_imag_q3_1['1'])/s_imag.shots

print('expect_value_real_q3_0 ', expect_value_real_q3_0)
print('expect_value_real_q3_1 ', expect_value_real_q3_1)
print('expect_value_imag_q3_0 ', expect_value_imag_q3_0)
print('expect_value_imag_q3_1 ', expect_value_imag_q3_1)

expect_value_real_q3_0  0.39304161071777344
expect_value_real_q3_1  0.37724876403808594
expect_value_imag_q3_0  -0.001209259033203125
expect_value_imag_q3_1  0.000179290771484375


In [9]:
# this is what bachelors have
print('Theoretical |0> = {:.4f} , |1> = {:.4f}'.format(n2(np.conj(a)*c+np.conj(b)*e), n2(np.conj(a)*d+np.conj(b)*f)))
print('Simulation  |0> = {:.4f} , |1> = {:.4f}'.format(expect_value_real_q3_0, expect_value_real_q3_1))

Theoretical |0> = 0.3937 , |1> = 0.3759
Simulation  |0> = 0.3930 , |1> = 0.3772


In [10]:
# this is what it should be (just realized that the imaginary part needs to be 0 -.-)
expect_value_q3_0 = expect_value_real_q3_0 + expect_value_imag_q3_0*1j
expect_value_q3_1 = expect_value_real_q3_1 + expect_value_imag_q3_1*1j

coef0 = np.absolute(np.sqrt(expect_value_q3_0))**2
coef1 = np.absolute(np.sqrt(expect_value_q3_1))**2

print('Theoretical |0> = {:.4f} , |1> = {:.4f}'.format(n2(np.conj(a)*c+np.conj(b)*e), n2(np.conj(a)*d+np.conj(b)*f)))
print('Simulation  |0> = {:.4f} , |1> = {:.4f}'.format(coef0, coef1))

Theoretical |0> = 0.3937 , |1> = 0.3759
Simulation  |0> = 0.3930 , |1> = 0.3772
